In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score

In [4]:
path_train = r"datasets/train.csv"
path_test  = r"datasets/test.csv"
path_add   = r"datasets/addresses.csv"
path_ll    = r"datasets/latlons.csv"

In [7]:
df_train = pd.read_csv(path_train, encoding="ISO-8859-1", engine='python')
df_test  = pd.read_csv(path_test, encoding="ISO-8859-1", engine='python')
df_add   = pd.read_csv(path_add, encoding="ISO-8859-1", engine='python').set_index('address')
df_ll    = pd.read_csv(path_ll, encoding="ISO-8859-1", engine='python').set_index('address')

In [8]:
df_add = df_add.join(df_ll, how="left")
df_add = df_add.set_index('ticket_id')

In [9]:
useless_labels = ['agency_name', 'violation_street_number', 'mailing_address_str_number', 'state', 
                  'disposition', 'violator_name', 'zip_code', 'country', 'city','inspector_name',
                  'violation_street_name', 'violation_zip_code', 'violation_description', 
                  'mailing_address_str_name', 'non_us_str_code', 'ticket_issued_date', 
                  'hearing_date', 'violation_code', 'grafitti_status']
    
    
train_leaks = [
        'balance_due',
        'collection_status',
        'compliance_detail',
        'payment_amount',
        'payment_date',
        'payment_status', 
    ]

In [10]:
df_train.drop(useless_labels,inplace = True,axis = 1)
df_test.drop(useless_labels,inplace = True,axis = 1)
df_train = df_train[(df_train.compliance == 1.0) | (df_train.compliance == 0.0)]
df_train.drop(train_leaks,inplace = True,axis = 1)

/home/ketan/miniconda3/envs/Project1/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]
X_train = X_train.set_index('ticket_id')
X_train = X_train.join(df_add,how = 'left')

In [12]:
X_test = df_test
X_test = X_test.set_index('ticket_id')
X_test = X_test.join(df_add,how = 'left')

In [13]:
X_train.lat.fillna(method='pad', inplace=True)
X_train.lon.fillna(method='pad', inplace=True)
X_test.lat.fillna(method='pad', inplace=True)
X_test.lon.fillna(method='pad', inplace=True)

In [14]:
param_grid = {'C':[0.01,0.1,1,10,100]}

In [15]:
s = MinMaxScaler()
X_train_scaled = s.fit_transform(X_train)
X_test_sclaed = s.transform(X_test)

In [16]:
grid_cv = GridSearchCV(LogisticRegression(),param_grid=param_grid,cv = 5,scoring='roc_auc')
grid_cv.fit(X_train_scaled,y_train)

/home/ketan/miniconda3/envs/Project1/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ketan/miniconda3/envs/Project1/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/s

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100]}, scoring='roc_auc')

In [17]:
y = grid_cv.predict_proba(X_test_sclaed)[:,1]   
result = X_test.index
result = pd.Series(data = y,index = result,name = 'compliance')

In [18]:
result.head()

ticket_id
284932    0.068681
285362    0.011564
285361    0.078909
285338    0.077714
285346    0.095258
Name: compliance, dtype: float64